In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [34]:
diabetes = pd.read_csv("diabetes_prediction_dataset.csv")

diabetes = diabetes[["gender", "Age", "BMI", "blood_glucose_levels"]]

diabetes

,gender,Age,BMI,blood_glucose_levels
0,Female,80.0,27.32,90
1,Female,67.0,27.32,80
2,Female,58.0,27.32,90
3,Male,2.0,18.41,100
4,Male,26.0,27.32,100
...,...,...,...,...
49996,Female,80.0,27.32,90
49997,Female,2.0,17.37,100
49998,Male,66.0,27.83,155
49999,Female,24.0,35.42,100


In [36]:
diabetes_tidied = pd.read_csv("diabetes_tidied.csv")

In [37]:
diabetes_train, diabetes_test = train_test_split(diabetes_tidied, train_size=.75)

In [38]:
diabetes_train.to_csv("diabetes_tidied_train.csv")

In [39]:
diabetes_test.to_csv("diabetes_tidied_test.csv")